In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
api_emb = np.load('api_emb.npy')
app_emb = np.load('app_tr_emb.npy') # (# of apps, 64)

bm_tr = np.load('bm_tr.npy') # (# of tr apps, 1000 apis)
counts_tr = np.load('counts_tr.npy')

In [4]:
bm_tr.shape, api_emb.shape

((1335, 1000), (1000, 64))

In [5]:
X_train, X_val, y_train, y_val = train_test_split(bm_tr, app_emb)

In [6]:
device = torch.device('cuda:0')

In [7]:
X_train = torch.tensor(X_train, dtype=torch.float).to(device)
X_val = torch.tensor(X_val, dtype=torch.float).to(device)
y_train = torch.tensor(y_train, dtype=torch.float).to(device)
y_val = torch.tensor(y_val, dtype=torch.float).to(device)

In [23]:
class CustomNet(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(CustomNet, self).__init__()
        self.w = torch.nn.Linear(64, 64)
        self.api_emb = torch.tensor(api_emb, dtype=torch.float, device=device)
        
    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        # app (1, 1000) 0 0 -10 15, 0 0 ...
        masked_apis = x.matmul(self.api_emb).float() # (1, 1000) * (1000, 64)
        y_pred = self.w(masked_apis) # (1, 64) (64, 64) - > (1, 64)
        
        return y_pred

In [24]:
model = CustomNet().to(device)
criterion = torch.nn.MSELoss(reduction='mean')

In [72]:
learning_rate = 1e-4
optimizer = torch.optim.Adamax(model.parameters(), lr=learning_rate)
for t in range(20000):
    y_pred = model(X_train)

    # Compute and print loss
    loss = criterion(y_pred, y_train)
#     if t % 100 == 99:
#         print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if t % 1000 == 999:
        with torch.no_grad():
            valid_loss = criterion(model(X_val), y_val)
            print(t, loss.item(), valid_loss.item())

999 40.334632873535156 53.03116989135742
1999 21.73644256591797 28.106674194335938
2999 11.909311294555664 15.496296882629395
3999 6.110782623291016 8.305177688598633
4999 2.68356990814209 3.8672738075256348
5999 0.8778526186943054 1.3372737169265747
6999 0.21122288703918457 0.32382649183273315
7999 0.06898479163646698 0.0918632447719574
8999 0.056270793080329895 0.07144784927368164
9999 0.0558808371424675 0.07096283882856369
10999 0.05587274953722954 0.07095855474472046
11999 0.05587157979607582 0.07096091657876968
12999 0.05587238073348999 0.07097017019987106
13999 0.05587095022201538 0.07096712291240692
14999 0.05587247759103775 0.07095225900411606
15999 0.05587132275104523 0.07095859199762344
16999 0.05587311089038849 0.07096146792173386
17999 0.05587209388613701 0.07096107304096222
18999 0.05587111413478851 0.07096115499734879
19999 0.05587109550833702 0.07095973938703537


In [73]:
bm_tst = np.load('bm_tst.npy')
bm_tst = torch.tensor(bm_tst, dtype=torch.float).to(device)

In [74]:
app_tst_emb = model.forward(bm_tst).cpu().detach().numpy()

In [75]:
import pandas as pd

In [76]:
meta_tr = pd.read_csv('../data/processed/meta_tr.csv', index_col=0)
meta_tst = pd.read_csv('../data/processed/meta_tst.csv', index_col=0)

In [77]:
y_tr = meta_tr.label == 'class1'
y_tst = meta_tst.label == 'class1'

In [78]:
from sklearn.svm import SVC
svm = SVC(kernel='linear')

In [79]:
svm.fit(app_emb, y_tr)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [80]:
svm.score(app_emb, y_tr)

0.9782771535580524

In [81]:
svm.score(app_tst_emb, y_tst)

0.9235955056179775